In [385]:
import numpy as np

In [386]:
#model environment parameters
adjustment_interval_min = 15
charging_duration_min = 120
min_charging_rate_kW = 0
max_charging_rate_kW = 22
battery_capacity_kWh = 32

expected_usage_kWh = 30
sigma_kWh = 5

#optimizer (Reinforcement Learner) parameters
fully_exhousted_penalty = -10_000_000_000 #penalty for charging can get very high itself (-329_703_933)

#computed parameters
num_charging_adjustments = int(charging_duration_min/adjustment_interval_min)


A = np.ones(num_charging_adjustments)

print(num_charging_adjustments)
print(A)

8
[1. 1. 1. 1. 1. 1. 1. 1.]


In [387]:
samples = np.random.normal(expected_usage_kWh, sigma_kWh, 10)
desired_charge = 30
print(samples)

[25.02703799 30.00857245 30.47460021 38.09894709 30.93160914 29.84633147
 42.92808675 26.38080986 26.82751994 24.1494787 ]


In [392]:
def calcChargingCost(t, p):
    e = np.exp(1)
    return A[t]*(e**p)

def rewardF(totalChargeByInterval, requiredkWh):
    totalCharged = totalChargeByInterval/(int(60/adjustment_interval_min))
    # diviation penalty
    divitation = abs(requiredkWh - totalCharged)
    divitationCost = max(divitation, .001)
    # extra penalty for running out
    runOutCost = fully_exhousted_penalty if requiredkWh > totalCharged else 0
    return  divitationCost + runOutCost

print(rewardF(120, 30))
print(rewardF(110, 30))

0.001
-9999999997.5


In [389]:
#generate env

def convertActionToChargingPower(value):
    if value == 0:
        return 0
    if value == 1:
        return 7
    if value == 2:
        return 14
    if value == 3:
        return 21

class env:
    def __init__(self):
        self.nA = 4
        self.nS = self.totalNodes(num_charging_adjustments)
        self.P = self.create_P()
        """ 
        print(self.nS)
        print(self.nA)
        print(self.P) """

    def totalNodes(self, layer):
        totalNodes = 0
        for lvl in range(0, layer):
            totalNodes = totalNodes + self.nA**lvl
        return totalNodes

    def create_P(self):
        P = []

        numberOfActions = self.nA
        
        def getActionValues(level, levelId):
            actionValue = levelId%numberOfActions
            values = [actionValue]
            ids = [levelId]
            for lvl in range(0, level):
                lastActionId = ids[0]//numberOfActions
                lastActionValue = lastActionId%numberOfActions
                values.insert(0, lastActionValue)
                ids.insert(0, lastActionId)
            return values

        actions = list()

        states = list()

        print("Actions", self.totalNodes(num_charging_adjustments))

        for actionLevel in range(0, num_charging_adjustments + 1):#+1 because the root of the tree has no pretaken action
            lastLayerActions = self.totalNodes(actionLevel)
            layerActions = self.totalNodes(actionLevel+1)

            states.append([])

            for levelActionId in range(0, numberOfActions**actionLevel):
                # levelActionId:
                #0 0 0 0
                #  1 1 1
                #    2 2
                #      3
                #
                # globalIds:
                #0 1 3 6
                #  2 4 7
                #    5 8
                #      9
                #

                #construct location in state tree
                action = levelActionId%numberOfActions
                globalActionId = (lastLayerActions-1) + levelActionId
                actions.append(0)

                originStateLevelId = levelActionId//numberOfActions
                originStateId = originStateLevelId+self.totalNodes(actionLevel-1)
                destinationStateId = self.totalNodes(actionLevel)+(originStateLevelId*numberOfActions)+action

                if(originStateId == len(P)):
                    P.append([])
                elif(originStateId > len(P)):
                    print("!Skipped state!")


                if(actionLevel != num_charging_adjustments):
                    #get total charging power
                    actionValues = getActionValues(actionLevel, levelActionId)
                    #remove root node
                    actionValues = actionValues[1:]
                    charge = 0
                    for i, action in enumerate(actionValues):
                        charge = charge + convertActionToChargingPower(action)
                    rewardValue = 0
                    #limit charge to battery capacity
                    if(charge/4 < battery_capacity_kWh):
                        rewardValue = -(calcChargingCost(actionLevel - 1, convertActionToChargingPower(action))/4)

                    actionTransition = ( 1, destinationStateId, int(rewardValue), False)
                    actions[globalActionId] = actionTransition
                    P[originStateId].append([actionTransition])
                else:
                    #get total charging power
                    actionValues = getActionValues(actionLevel, levelActionId)
                    #remove root node
                    actionValues = actionValues[1:]
                    charge = 0
                    for i, action in enumerate(actionValues):
                        charge = charge + convertActionToChargingPower(action)
                    rewardValue = 0
                    #limit charge to battery capacity
                    if(charge/4 < battery_capacity_kWh):
                        rewardValue = -(calcChargingCost(actionLevel - 1, convertActionToChargingPower(action))/4)
                    rewardValue = rewardF(charge, desired_charge) + rewardValue
                    
                    actionTransition = (1, destinationStateId, int(rewardValue), True)
                    actions[globalActionId] = actionTransition
                    P[originStateId].append([actionTransition])
        #remove root node
        P[0] = P[0][1:]
        return P

env = env()

Actions 21845


## Example Decision Path

In [391]:
# P[n] is the list of states
# each state containes a list of actions P[n][a]
# each action contains a list of tuples like (transition_prob, next_state, reward, done)
# since transition_prob is always 1 there is also only one tuple per action

# take example path

# charge_level_test = [3,3,3,3,3,3,3,3] # This will stop giving negative rewards after some actions when the battery is full
charge_level_test = [2,2,2,2,2,2,2,2] # This will have a very large penalty in the last action when the needed output is not met
# charge_level_test = [2,2,2,3,3,2,2,2] # This will have lower penalty in the end

for prob, next_state, reward, done in env.P[0][charge_level_test[0]]:
    print("1. Action: ", prob, next_state, reward, done)
    for prob, next_state, reward, done in env.P[next_state][charge_level_test[1]]:
        print("2. Action: ", prob, next_state, reward, done)
        for prob, next_state, reward, done in env.P[next_state][charge_level_test[2]]:
                print("3. Action: ", prob, next_state, reward, done)
                for prob, next_state, reward, done in env.P[next_state][charge_level_test[3]]:
                        print("4. Action: ", prob, next_state, reward, done)
                        for prob, next_state, reward, done in env.P[next_state][charge_level_test[4]]:
                                print("5. Action: ", prob, next_state, reward, done)
                                for prob, next_state, reward, done in env.P[next_state][charge_level_test[5]]:
                                        print("6. Action: ", prob, next_state, reward, done)
                                        for prob, next_state, reward, done in env.P[next_state][charge_level_test[6]]:
                                                print("7. Action: ", prob, next_state, reward, done)
                                                for prob, next_state, reward, done in env.P[next_state][charge_level_test[7]]:
                                                    print("8. Action: ", prob, next_state, reward, done)


1. Action:  1 3 -300651 False
2. Action:  1 15 -300651 False
3. Action:  1 63 -300651 False
4. Action:  1 255 -300651 False
5. Action:  1 1023 -300651 False
6. Action:  1 4095 -300651 False
7. Action:  1 16383 -300651 False
8. Action:  1 65535 -10000300649 True
